In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchsummary import summary
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.datasets import mnist

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [131]:
# 使用 keras 直接載入 MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalization
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 256

# 利用 sklearn，將每一類都以 8:2 的比例分成訓練資料和測試資料
features_train, features_test, targets_train, targets_test = train_test_split(train_images, train_labels, test_size = 0.2, random_state = 42)

In [4]:
features_train[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [5]:
targets_train[0]

5

In [6]:
# 將切好的 data 轉成 tensor 形式
# Training Datasets
featuresTrain = torch.from_numpy(features_train)
targetsTrain = torch.from_numpy(targets_train).type(torch.LongTensor)     # data type is long

In [7]:
type(featuresTrain)

torch.Tensor

In [8]:
featuresTrain.dtype

torch.float32

In [9]:
featuresTrain.shape

torch.Size([48000, 28, 28])

In [10]:
targetsTrain

tensor([5, 0, 1,  ..., 0, 0, 4])

In [11]:
# Testing Datasets
featuresTest = torch.from_numpy(features_test)
targetsTest = torch.from_numpy(targets_test).type(torch.LongTensor)       # data type is long

In [12]:
# 使用 torch.utils.data.TensorDataset 將 train 和 test datasets 存成 tensor 形式
# Pytorch train and test TensorDataset
train = torch.utils.data.TensorDataset(featuresTrain, targetsTrain)
test = torch.utils.data.TensorDataset(featuresTest, targetsTest)

In [13]:
type(train)

torch.utils.data.dataset.TensorDataset

In [14]:
LR = 0.01                                                        # Learning Rate
batch_size = 100                                                 # Batch size
n_iters = 10000                                                  # Iterations each epoch
num_epochs = n_iters / (len(features_train) / batch_size)        
num_epochs = int(num_epochs)                                     # Epochs

In [15]:
num_epochs

20

In [16]:
# torch.utils.data.TensorDatasetDataLoader(dataset, batch_size=1, shuffle=False,...) 為數據加載器
# 組合數據集和採樣器，並在數據集上提供單進程或多進程迭代器
# Pytorch DataLoader
train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False)

In [111]:
# Create CNN Model
class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        cnn1_out_ch = 4
        cnn2_out_ch = 4
        have_linear = False
        # Convolution 1 , input_shape=(1,28,28), output_shape=(1,26,26)
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=cnn1_out_ch, kernel_size=3, stride=1, padding=0)
        # activation
        self.relu1 = nn.ReLU() 
        # Max pool 1, output_shape=(1,13,13)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2) 
        # Convolution 2, output_shape=(1,11,11)
        self.cnn2 = nn.Conv2d(in_channels=cnn1_out_ch, out_channels=cnn2_out_ch, kernel_size=3, stride=1, padding=0)
        # activation
        self.relu2 = nn.ReLU() 
        # Convolution 3, output_shape=(1,9,9)
        self.cnn3 = nn.Conv2d(in_channels=cnn2_out_ch, out_channels=10, kernel_size=3, stride=1, padding=0)
        # activation
        self.relu3 = nn.ReLU() 
        # Max pool 2, output_shape=(10,4,4)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        # Average pool, output_shape=(10,1,1)
        self.avgpool = nn.AvgPool2d(kernel_size=4)
        # Fully connected 1, input_shape=(1*5*5)
        # self.fc1 = nn.Linear(10 * 5 * 5, 10) 
        if have_linear:
            self.fc1 = nn.Linear(10 * 1 * 1, 10) 
    
    def forward(self, x):
        have_linear = False
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        # Max pool 1
        out = self.maxpool1(out)
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        # Convolution 3 
        out = self.cnn3(out)
        out = self.relu3(out)
        # Max pool 2 
        out = self.maxpool2(out)
        # Average pool
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        # print(out.size())
        # Linear function (readout)
        if have_linear:
            out = self.fc1(out)
        return out

model = CNN_Model().to(device)                            # Create the CNN Model             
optimizer = torch.optim.Adam(model.parameters(), lr = LR) # 選擇你想用的 optimizer(Adam)
summary(model, (1, 28, 28))                               # 利用 torchsummary 的 summary package 印出模型資訊，input size: (1 * 28 * 28) 
loss_func = nn.CrossEntropyLoss()                         # 選擇想用的 loss function(CrossEntropy)
input_shape = (-1, 1, 28, 28)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 26, 26]              40
              ReLU-2            [-1, 4, 26, 26]               0
         MaxPool2d-3            [-1, 4, 13, 13]               0
            Conv2d-4            [-1, 4, 11, 11]             148
              ReLU-5            [-1, 4, 11, 11]               0
            Conv2d-6             [-1, 10, 9, 9]             370
              ReLU-7             [-1, 10, 9, 9]               0
         MaxPool2d-8             [-1, 10, 4, 4]               0
         AvgPool2d-9             [-1, 10, 1, 1]               0
Total params: 558
Trainable params: 558
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.07
Params size (MB): 0.00
Estimated Total Size (MB): 0.07
---------------------------------------------------

In [112]:
# 訓練 function
def fit_model(model, loss_func, optimizer, input_shape, num_epochs, train_loader, test_loader):
    # Traning the Model
    # 儲存訓練資訊的 List
    training_loss, training_accuracy = [], []
    validation_loss, validation_accuracy = [], []
    for epoch in range(num_epochs):
        # ---------------------------
        # Training Stage
        # ---------------------------
        correct_train, total_train = 0, 0
        for i, (images, labels) in enumerate(train_loader):
            train, labels = images.view(input_shape).to(device), labels.to(device)  # extract training data and label
            optimizer.zero_grad()                                                   # reset gradient
            outputs = model(train)                                                  # 將訓練資料輸入至模型進行訓練 (Forward propagation)
            train_loss = loss_func(outputs, labels)                                 # 計算 loss
            train_loss.backward()                                                   # 將 loss 反向傳播
            optimizer.step()                                                        # 更新權重
            
            # 計算訓練資料的準確度 (correct_train / total_train)
            predicted = torch.max(outputs.data, 1)[1]                               # 取出預測的 maximum
            total_train += len(labels)                                              # 全部的 label 數 (Total number of labels)
            correct_train += (predicted == labels).float().sum()                    # 全部猜中的個數 (Total correct predictions)
        
        # 將 accuracy 和 loss 存入 list
        train_accuracy = 100 * correct_train / float(total_train)                   # training accuracy (To cpu())
        training_accuracy.append(train_accuracy.cpu())
        training_loss.append(train_loss.data.cpu())                                 # training loss (To cpu())

        # --------------------------
        # Testing Stage
        # --------------------------
        correct_test, total_test = 0, 0
        for images, labels in test_loader:
            test, labels = images.view(input_shape).to(device), labels.to(device)   # 取出 testing data 以及 labels(轉 device 的型態)
            outputs = model(test)                                                   # 將測試資料輸入至模型進行測試 (Forward propagation)
            val_loss = loss_func(outputs, labels)                                   # 計算 loss
            
            # 計算測試資料的準確度 (correct_test / total_test)
            predicted = torch.max(outputs.data, 1)[1]                               # 取出預測的 maximum
            total_test += len(labels)                                               # 全部的 label 數 (Total number of labels)
            correct_test += (predicted == labels).float().sum()                     # 全部猜中的個數 (Total correct predictions)
            
        # 將 accuracy 和 loss 存入 list
        val_accuracy = 100 * correct_test / float(total_test)                       # testing accuracy (To cpu())
        validation_accuracy.append(val_accuracy.cpu())
        validation_loss.append(val_loss.data.cpu())                                 # testing loss (To cpu())
        
        # 顯現當前 Epoch 訓練情況
        print('Train Epoch: {}/{} Traing_Loss: {} Traing_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy, val_loss.data, val_accuracy))
    return training_loss, training_accuracy, validation_loss, validation_accuracy

In [113]:
training_loss, training_accuracy, validation_loss, validation_accuracy = fit_model(model, loss_func, optimizer, input_shape, num_epochs, train_loader, test_loader)

Train Epoch: 1/20 Traing_Loss: 0.6322295665740967 Traing_acc: 60.568748% Val_Loss: 0.6495568752288818 Val_accuracy: 80.241669%
Train Epoch: 2/20 Traing_Loss: 0.4600367844104767 Traing_acc: 81.631248% Val_Loss: 0.5719389915466309 Val_accuracy: 83.875000%
Train Epoch: 3/20 Traing_Loss: 0.38932469487190247 Traing_acc: 83.935417% Val_Loss: 0.5516255497932434 Val_accuracy: 85.141670%
Train Epoch: 4/20 Traing_Loss: 0.3769429326057434 Traing_acc: 85.502083% Val_Loss: 0.5288537740707397 Val_accuracy: 86.574997%
Train Epoch: 5/20 Traing_Loss: 0.3445349931716919 Traing_acc: 86.895836% Val_Loss: 0.5148516893386841 Val_accuracy: 87.558334%
Train Epoch: 6/20 Traing_Loss: 0.3144491910934448 Traing_acc: 87.793747% Val_Loss: 0.49560168385505676 Val_accuracy: 88.091667%
Train Epoch: 7/20 Traing_Loss: 0.3027723431587219 Traing_acc: 88.477081% Val_Loss: 0.47782889008522034 Val_accuracy: 88.308334%
Train Epoch: 8/20 Traing_Loss: 0.288073867559433 Traing_acc: 88.929169% Val_Loss: 0.47226154804229736 Val_ac

In [114]:
model.eval()
for name, param in model.named_parameters():
    print(name)
    print(param.data)

cnn1.weight
tensor([[[[-0.1552, -1.5892, -1.4030],
          [ 0.7387, -0.6188, -0.2291],
          [ 0.6840,  1.7093,  1.2374]]],


        [[[ 2.9713,  1.8625,  0.7104],
          [ 1.7667,  0.3682,  0.7768],
          [ 0.2018, -0.0494,  0.0713]]],


        [[[ 0.8434,  0.6965,  0.6157],
          [-0.9713,  0.6163,  0.0070],
          [ 0.7122,  0.9946,  1.7027]]],


        [[[ 1.4453, -0.8497, -1.8522],
          [ 1.8171,  0.1701, -2.1512],
          [ 0.9111,  1.7062, -0.4916]]]])
cnn1.bias
tensor([-0.0770,  0.0230, -0.0263, -0.3626])
cnn2.weight
tensor([[[[ 1.6668e+00,  4.0573e-02, -1.7494e-01],
          [ 6.4909e-01, -8.9866e-01, -6.7957e-01],
          [ 4.7733e-01, -1.5891e+00,  2.1699e+00]],

         [[-4.1872e-01, -1.0789e-01,  1.1632e-01],
          [ 1.1532e+00,  1.7145e-01, -3.0470e-02],
          [-1.1479e-01, -1.1646e-01,  4.3349e-01]],

         [[-5.9086e-01, -3.7536e-01,  8.7902e-02],
          [ 5.5081e-01,  3.4715e-01,  1.8320e-01],
          [-1.4742e-01,  1

In [132]:
# test with test data
testImages = torch.from_numpy(test_images)
testLabels = torch.from_numpy(test_labels).type(torch.LongTensor)   
verify = torch.utils.data.TensorDataset(testImages, testLabels)
verify_loader = torch.utils.data.DataLoader(verify, batch_size = batch_size, shuffle = False)

In [116]:
def verify_model(model, input_shape, loader):
    correct_test, total_test = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            test, labels = images.view(input_shape).to(device), labels.to(device)   # 取出 testing data 以及 labels(轉 device 的型態)
            outputs = model(test)                                                   # 將測試資料輸入至模型進行測試 (Forward propagation)                                 # 計算 loss
            
            # 計算測試資料的準確度 (correct_test / total_test)
            print(torch.max(outputs.data, 1)[1])
            print(labels)
            print("--------------")
            predicted = torch.max(outputs.data, 1)[1]                               # 取出預測的 maximum
            total_test += len(labels)                                               # 全部的 label 數 (Total number of labels)
            correct_test += (predicted == labels).float().sum()   
    val_accuracy = 100 * correct_test / float(total_test)
    print("Verified Accuracy: %f (%d/%d)" % (val_accuracy, correct_test, total_test))
    return

In [133]:
verify_model(model, input_shape, verify_loader)

tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 5, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 2, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 9,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0,
        2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 3, 3, 1, 8,
        1, 7, 0, 9])
tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0,
        2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4,
        1, 7, 6, 9])
--------------
tensor([6, 0, 5, 4, 2, 9, 2, 1, 9, 4, 8, 1, 3, 9, 7, 4, 4, 4, 9, 2, 5, 6, 7, 6,
        9, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2,
        0, 3, 9, 5, 5, 5, 1, 5, 6, 0, 5, 9, 4, 6, 5, 4, 6, 5, 8, 3, 1, 4, 4, 7,
        2, 3, 2, 1, 1, 8, 1, 8, 1, 8, 5, 0, 3, 9, 2, 5, 0, 1, 1

In [121]:
torch.save(model, "model_90.pt",_use_new_zipfile_serialization=True)

In [127]:
from PIL import Image
import numpy as np

In [123]:

img = Image.open("pics/file_6_1.png")
img.load()
img = np.asarray( img, dtype="int32" )
img = img.astype('float32') / 255
img = torch.from_numpy(img)
img = img.unsqueeze(0)

In [124]:
out = model(img)

In [125]:
print(out.size())
# out = out.squeeze(1)
# print(out.size())
print(out)
torch.max(out, 0)

torch.Size([10, 1])
tensor([[ 1.2902],
        [11.7298],
        [ 3.9061],
        [ 7.7338],
        [ 2.3796],
        [ 0.6430],
        [ 3.1881],
        [ 0.6020],
        [ 1.9333],
        [ 6.8049]], grad_fn=<ViewBackward0>)


torch.return_types.max(
values=tensor([11.7298], grad_fn=<MaxBackward0>),
indices=tensor([1]))

In [176]:
torch.max(out, 0)[1]

tensor([1])